<h2>Question 1<\h2>
<h4>The reason<\h4>

In [13]:
%pylab

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


/home/xiongti1/anaconda3/envs/cv2/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['product', 'correlate', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
import cv2
from helper import *
from itertools import product
from tqdm import tqdm
from numba import jit

In [15]:
left = cv2.imread("A4_files/000020_left.jpg").astype(float)/255
right = cv2.imread("A4_files/000020_right.jpg").astype(float)/255
with open("A4_files/000020.txt") as file:
    x_min, y_min, x_max, y_max = [float(i) for i in file.read().split()[1:]]
with open("A4_files/000020_allcalib.txt") as file:
    f, px, py, baseline = [float(i) for i in file.read().split()[1::2]]
print(x_min, y_min, x_max, y_max, f, px, py, baseline)

685.05 181.43 804.68 258.21 721.5377 609.5593 172.854 0.5327119288


In [16]:
patchsize = 5
imshow(left)

In [17]:
tar = left[int(y_min):int(y_max), int(x_min):int(x_max)]
tar = np.pad(tar, [[patchsize//2, patchsize//2], [patchsize//2, patchsize//2], [0, 0]])
imshow(tar)

### Q2
a. I am using SSD for this part

In [18]:
@jit(nopython=True, fastmath=True)
def SSD(patch1, patch2):
    return np.sum((patch1 - patch2)**2)

@jit(nopython=True, fastmath=True)
def NC(patch1, patch2):
    return np.sum(patch1*patch2)/np.sqrt(np.sum(patch1**2))/np.sqrt(np.sum(patch2**2))

@jit(nopython=True)
def rowler(patch, imgr, row_idx):
    patchsize = patch.shape[0]
    row = imgr[row_idx:row_idx+patchsize]
    ssd_list = []
    for i in range(row.shape[1]-patchsize+1):
        ssd_list.append(NC(patch, imgr[row_idx:row_idx+patchsize, i:i+patchsize]))
    return np.array(ssd_list).T

@jit(nopython=True)
def disparity(imgl, imgr, y_l, x_l, patch_size, y_min):
    patch = imgl[y_l:y_l+patch_size, x_l:x_l+patch_size]
    row_idx = y_l
    ssd_list = rowler(patch, imgr, row_idx)
    x_r = ssd_list.argmax()
    return x_l - x_r 

@jit(nopython=True, parallel=True)
def img_disp(imgl, imgr, patch_size, x_min, y_min, x_max, y_max):
    dy = y_max - y_min
    dx = x_max - x_min
    new_img = np.zeros((dy-patch_size+1, dx-patch_size+1))
    for y in range(y_min, y_max-patch_size+1):
        for x in range(x_min, x_max-patch_size+1):
            new_img[y - y_min, x - x_min] = disparity(imgl, imgr, y, x, patch_size, y_min)
    return new_img

In [19]:
imgl = left
imgr = right
patch_size = 5

new_img = img_disp(imgl, imgr, patch_size, int(x_min), int(y_min), int(x_max), int(y_max))

In [20]:
imshow(new_img, cmap="jet")
colorbar()

In [21]:
imshow(f*baseline/new_img, cmap="jet")
colorbar()

/home/xiongti1/anaconda3/envs/cv2/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


In [22]:
rowler(tar[50:55, 50:55], imgr, 50+int(y_min)).max()

0.9817363062135014

In [23]:
deeppruner_result = cv2.imread("DeepPruner_result.png")

In [24]:
imshow(deeppruner_result[...,0], cmap="jet")

b. This Network has the following procedures:
- Feature Extractor

    The Feature Extractor create a pixel wide feature representation by downsampling networks. Creating a feature map that is 1/4 of the original input image size. 
    
- Differentiable Patch Match
    
    The differentiable patch matching is achieved by using a PatchMatch-based prunner that predict a confidence range for each pixel and construct a sparse cost volume. 
    The prunning module can be show as follows:
    1. Particle Sampling Layer:
       
       Generate disparity values for each pixel, from our search space. 
       
    2. Propagation Layer:
        
        
    
    3. Evaluation Layer:

- Confidence Range Prediction

    The Confidence Range Prediction is achieved 
    
    
- 3D Cost Aggregation 
- Refinement

### Q3

In [26]:
I1 = cv2.imread("./I1.jpg")
I2 = cv2.imread("./I2.jpg")
I3 = cv2.imread("./I3.jpg")